In [14]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

In [15]:
df = pd.read_csv('Data/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [16]:
string_col = df.select_dtypes(include="object").columns
df[string_col] = df[string_col].astype("string")
print(df.dtypes) # check if converted properly

Age                        int64
Sex               string[python]
ChestPainType     string[python]
RestingBP                  int64
Cholesterol                int64
FastingBS                  int64
RestingECG        string[python]
MaxHR                      int64
ExerciseAngina    string[python]
Oldpeak                  float64
ST_Slope          string[python]
HeartDisease               int64
dtype: object


In [17]:
# distribution of categorical values:
df[string_col].head()
for col in string_col:
    print(f"The distribution of categorical valeus in the {col} is : ")
    print(df[col].value_counts())

The distribution of categorical valeus in the Sex is : 
Sex
M    725
F    193
Name: count, dtype: Int64
The distribution of categorical valeus in the ChestPainType is : 
ChestPainType
ASY    496
NAP    203
ATA    173
TA      46
Name: count, dtype: Int64
The distribution of categorical valeus in the RestingECG is : 
RestingECG
Normal    552
LVH       188
ST        178
Name: count, dtype: Int64
The distribution of categorical valeus in the ExerciseAngina is : 
ExerciseAngina
N    547
Y    371
Name: count, dtype: Int64
The distribution of categorical valeus in the ST_Slope is : 
ST_Slope
Flat    460
Up      395
Down     63
Name: count, dtype: Int64


In [18]:
# label encoding of categorical variables (this is fine for random forest)
# Initialize a dictionary to store label encoders
label_encoders = {}

# Label encoding of categorical variables
for col in string_col:
    le = LabelEncoder() # initialize label encoder
    df[col] = le.fit_transform(df[col])
    # fit: During the fitting process, LabelEncoder learns the unique classes present in the data and assigns each class a unique integer.
    # transform: After learning the classes, it then transforms the original categorical values into their corresponding integer codes.
    label_encoders[col] = le

In [19]:
# Print the label encoding mapping for each categorical feature
for col in string_col:
    le = label_encoders[col]
    print(f"Label encoding for {col}:")
    for class_, label in zip(le.classes_, le.transform(le.classes_)):
        print(f"{class_} -> {label}")
    print()

Label encoding for Sex:
F -> 0
M -> 1

Label encoding for ChestPainType:
ASY -> 0
ATA -> 1
NAP -> 2
TA -> 3

Label encoding for RestingECG:
LVH -> 0
Normal -> 1
ST -> 2

Label encoding for ExerciseAngina:
N -> 0
Y -> 1

Label encoding for ST_Slope:
Down -> 0
Flat -> 1
Up -> 2



In [20]:
print(df[df['Cholesterol'] == 0].count())
df[df['Cholesterol'] == 0].head()

Age               172
Sex               172
ChestPainType     172
RestingBP         172
Cholesterol       172
FastingBS         172
RestingECG        172
MaxHR             172
ExerciseAngina    172
Oldpeak           172
ST_Slope          172
HeartDisease      172
dtype: int64


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
293,65,1,0,115,0,0,1,93,1,0.0,1,1
294,32,1,3,95,0,1,1,127,0,0.7,2,1
295,61,1,0,105,0,1,1,110,1,1.5,2,1
296,50,1,0,145,0,1,1,139,1,0.7,1,1
297,57,1,0,110,0,1,2,131,1,1.4,2,1


In [21]:
target_column = 'Cholesterol'
label_column = 'HeartDisease'

In [22]:
mask_missing = (df['Cholesterol'] == 0)

In [23]:
# Separate the features with missing values in the target column
mask_missing = df[target_column] == 0

# Replace 0 with NaN in the target column for easier handling
df[target_column].replace(0, np.nan, inplace=True)

# Impute missing values with the mean to perform KMeans clustering
features = df.drop(columns=[target_column, label_column])
imputer = SimpleImputer(strategy='mean')
features_imputed = pd.DataFrame(imputer.fit_transform(features), columns=features.columns)

# Include the target column without missing values for clustering
features_imputed[target_column] = df[target_column]
features_imputed.fillna(features_imputed.mean(), inplace=True)

# Perform KMeans clustering on rows without missing values in the target column
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(features_imputed[~mask_missing])

# Assign clusters to rows
df['Cluster'] = -1
df.loc[~mask_missing, 'Cluster'] = kmeans.predict(features_imputed[~mask_missing])

# Impute missing values in the target column using the cluster means
for cluster in range(kmeans.n_clusters):
    cluster_mean = df.loc[df['Cluster'] == cluster, target_column].mean()
    df.loc[(df['Cluster'] == cluster) & mask_missing, target_column] = cluster_mean

# Drop the 'Cluster' column
df.drop(columns='Cluster', inplace=True)

print(df)

     Age  Sex  ChestPainType  RestingBP  Cholesterol  FastingBS  RestingECG  \
0     40    1              1        140        289.0          0           1   
1     49    0              2        160        180.0          0           1   
2     37    1              1        130        283.0          0           2   
3     48    0              0        138        214.0          0           1   
4     54    1              2        150        195.0          0           1   
..   ...  ...            ...        ...          ...        ...         ...   
913   45    1              3        110        264.0          0           1   
914   68    1              0        144        193.0          1           1   
915   57    1              0        130        131.0          0           1   
916   57    0              1        130        236.0          0           0   
917   38    1              2        138        175.0          0           1   

     MaxHR  ExerciseAngina  Oldpeak  ST_Slope  Hear

/var/folders/x_/cp6k40b141nfh4rft4427nbm0000gn/T/ipykernel_40360/2865728659.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[target_column].replace(0, np.nan, inplace=True)


In [24]:
df[df[target_column] == 0]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [25]:
df.to_csv('Data/heart_kmeans_cleansed.csv', index=False)